### 第9回レポート課題その1

今回の実行分は前回のコードの続きに書いたが, 誤って前回分を削除してしまったので今回の部分のみをのせる.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import time
from google.colab import drive
drive.mount('/content/drive')

# 単層ニューラルネットワークを定義
class SingleLayerNN(nn.Module):
    def __init__(self, embedding_dim, num_labels):
        super(SingleLayerNN, self).__init__()
        self.linear = nn.Linear(embedding_dim, num_labels) #todo linear層を追加
    
    def forward(self, x):
        h1 = self.linear(x) #todo xをlinear層に入力
        return nn.Softmax(h1) #todo h1をsoftmaxしたものを返す

class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x) #todo データサイズを返す

    def __getitem__(self, idx):
        out_x = self.x[idx] #todo idx番目のxの要素を返す
        out_y = self.y[idx] #todo idx番目のyの要素を返す
        return out_x, out_y
        
torch.manual_seed(1)

embedding_dim = 300
num_labels = 4
NUM_EPOCH = 100

train_x, train_y = torch.load("/content/drive/My Drive/data/train_x.pt"), torch.load("/content/drive/My Drive/data/train_y.pt")
valid_x, valid_y = torch.load("/content/drive/My Drive/data/valid_x.pt"), torch.load("/content/drive/My Drive/data/valid_y.pt")

# データとラベルを1つにまとめる
dataset = MyDataset(train_x, train_y)

loss_fn = nn.CrossEntropyLoss()#todo 損失関数としてCrossEntropyLossを用いる


def train(model, train_loader):
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        pred = model.forward(data.type(torch.FloatTensor))  #todo dataをmodelに入力
        loss = loss_fn(pred, target)  #todo predとtargetと損失関数からロスを計算
        loss.backward() # 逆誤差伝搬を実施
        optimizer.step() # パラメータを更新

        
def evaluation(model, data, target):
    model.eval()
    with torch.no_grad():
        pred =  model(data.type(torch.FloatTensor)) #todo dataをmodelに入力
        loss = loss_fn(pred, target)  #todo predとtargetと損失関数からロスを計算
        pred_labels = torch.argmax(pred, axis=1)  #todo 推定したラベルを代入
        acc = (pred_labels == target).sum().item() / len(pred_labels) #todo pred_labelsとtargetから正解率を計算
    return loss.item(), acc


batch_size_list = [1, 2, 4, 8, 16, 32, 64, 128, 256] #todo バッチサイズの値のリスト．1,2,4,8,…が入る

for batch_size in batch_size_list:
    model = SingleLayerNN(embedding_dim, num_labels)
    optimizer = optim.SGD(model.parameters(), lr=0.1)

    # ミニバッチを扱うためのデータローダを作成
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(NUM_EPOCH):
        start_time = time.time() # 学習開始時の時間を取得
        train(model, train_loader)
        took_time = time.time() - start_time #todo 学習にかかった時間を代入

        valid_loss, valid_acc = evaluation(model, valid_x, valid_y)

        if epoch % 10 == 0:
            print(f"batch size:{batch_size}\tepoch: {epoch}")
            print(f"train time: {took_time}")
            print(f"<valid>\tloss: {valid_loss}\tacc: {valid_acc}")
    
    print("\n================\n")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


batch size:1	epoch: 0
train time: 3.4327142238616943
<valid>	loss: 0.9587206244468689	acc: 0.800599700149925
batch size:1	epoch: 10
train time: 3.3923325538635254
<valid>	loss: 0.9397221803665161	acc: 0.8013493253373314
batch size:1	epoch: 20
train time: 3.4230799674987793
<valid>	loss: 0.9378631114959717	acc: 0.8020989505247377
batch size:1	epoch: 30
train time: 3.4563117027282715
<valid>	loss: 0.936996340751648	acc: 0.8035982008995503
batch size:1	epoch: 40
train time: 3.40429949760437
<valid>	loss: 0.9364750981330872	acc: 0.802848575712144
batch size:1	epoch: 50
train time: 3.429076671600342
<valid>	loss: 0.9360454082489014	acc: 0.8020989505247377
batch size:1	epoch: 60
train time: 3.393639326095581
<valid>	loss: 0.8930366635322571	acc: 0.8538230884557722
batch size:1	epoch: 70
train time: 3.289168357849121
<valid>	loss: 0.8867972493171692	acc: 0.8590704647676162
batch size:1	epoch: 80
train time: 3.3845012187957764
<valid>	loss: 0.885372519493103	acc: 0.8598200899550225
batch size:

バッチサイズが大きくなるほど計算にかかる時間が短くなっていることが分かる.

#### 感想
穴埋めはPytorchの復習にもなり, 前回よりもやりやすかった. ただ細かいエラーの対処に時間がかかったので今後の課題だと感じた.